In [1]:


#vit finetuned

# new agumentation

#regularization loss for training only

# using 7 classes

# using cls token

# no mixed precision training

# kan-moe+attention gating(softmax)

# changing Activation to GELU() and MISH()

# using AdamW optimizer

# using vectorized operations and parallelized expert aggregation

# using learning rate scheduler CosineAnnealingLR:

# using Gradient clipping clip_grad_norm_: 
#--------------------------------------------------------------------------------

#more fine tuning ---
#more hidden dimensions----
#change attention mechanism in gating (adaptive attention)---------------
#more kan layers-----
#change optimizer to Lion and RAdam------
#change loss fuction to focal loss----
#change agumentation techniques-------
#use oneCycleLr-----------
#change activation to mish----------


In [2]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch
import timm
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
from torchvision.models import vit_b_16, ViT_B_16_Weights
from torch.utils.data import DataLoader
import numpy as np

from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn.utils import clip_grad_norm_
from timm import create_model
from torch.nn.modules.loss import CrossEntropyLoss
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

class MalariaDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        return image, label

    def classes(self):
        return self.data.classes

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),#for vit
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

import torchvision.transforms as transforms

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.Resize((224, 224)),#for vit
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])

])

# train_transforms = transforms.Compose([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),
#     transforms.RandomRotation(180),
#     transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
#     transforms.Resize((224, 224)),#for vit
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image

# ])

# Define directories
train_dir = '/teamspace/studios/this_studio/train_2023'
val_dir = '/teamspace/studios/this_studio/test_2023'

# Create datasets
train_data = MalariaDataset(train_dir,transform=train_transforms)
val_data = MalariaDataset(val_dir,transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True,num_workers=4,pin_memory=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False,num_workers=4,pin_memory=True)


#------------------

In [3]:

# -----------------------------
# Define Alternative Activations
# -----------------------------
class Mish(nn.Module):
    def forward(self, x):
        return x * torch.tanh(F.softplus(x))

# -----------------------------
# Vectorized KAN Layer using Grouped Convolutions
# -----------------------------
class VectorizedKANLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout=0.1, activation='GELU'):
        super(VectorizedKANLayer, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        # Grouped convolution to simulate separate linear layers per input channel.
        # First transformation: for each of the input_dim channels, produce hidden_dim outputs.
        self.conv1 = nn.Conv1d(in_channels=input_dim, 
                               out_channels=input_dim * hidden_dim,
                               kernel_size=1, groups=input_dim)
        # Choose activation function
        if activation.lower() == 'gelu':
            self.act = nn.GELU()
        elif activation.lower() == 'mish':
            self.act = Mish()
        else:
            self.act = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        # Second transformation: map hidden_dim back to 1 for each channel.
        self.conv2 = nn.Conv1d(in_channels=input_dim * hidden_dim, 
                               out_channels=input_dim,
                               kernel_size=1, groups=input_dim)

    def forward(self, x):
        # x shape: [batch_size, input_dim]
        x = x.unsqueeze(2)  # reshape to [B, input_dim, 1]
        out = self.conv1(x)  # [B, input_dim*hidden_dim, 1]
        out = self.act(out)
        out = self.dropout(out)
        out = self.conv2(out)  # [B, input_dim, 1]
        # Sum over the input_dim channels to produce a scalar per sample.
        out = torch.sum(out, dim=1)  # shape: [B, 1]
        return out.squeeze(1)  # shape: [B]



In [4]:
# -----------------------------
# Kolmogorov-Arnold Network using VectorizedKANLayer
# -----------------------------
class VectorizedKolmogorovArnoldNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, dropout=0.1, activation='GELU'):
        
        super(VectorizedKolmogorovArnoldNetwork, self).__init__()
        self.layers = nn.ModuleList([
            VectorizedKANLayer(input_dim, hidden_dim, dropout, activation) 
            for _ in range(num_layers)
        ])
        # Final fully connected layer that maps the concatenated layer outputs to logits.
        self.final_layer = nn.Linear(num_layers, num_classes)

    def forward(self, x):
        # Collect outputs from each KAN layer.
        layer_outputs = [layer(x) for layer in self.layers]  # Each is [B]
        concatenated_outputs = torch.stack(layer_outputs, dim=1)  # shape: [B, num_layers]
        logits = self.final_layer(concatenated_outputs)  # [B, num_classes]
        return logits

# -----------------------------
# KAN Expert using the vectorized KAN network
# -----------------------------
class KANExpert(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, dropout=0.1, activation='GELU'):
        super(KANExpert, self).__init__()
        self.kan = VectorizedKolmogorovArnoldNetwork(input_dim, hidden_dim, num_layers, num_classes, dropout, activation)

    def forward(self, x):
        return self.kan(x)



In [5]:
# -----------------------------
# Attention Gating Network (unchanged)
# -----------------------------
class AttentionGatingNetwork(nn.Module):
    def __init__(self, input_dim, num_experts, dropout=0.1):
        super(AttentionGatingNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, num_experts),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        raw_weights = self.fc(x)  # shape: [B, num_experts]
        attention_scores = F.softmax(raw_weights, dim=1)
        return attention_scores

# -----------------------------
# KANMoE Module with Vectorized Expert Aggregation
# -----------------------------
class KANMoE(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_experts, num_classes, top_k=2, reg_lambda=0.01, dropout=0.1, activation='GELU'):
        super(KANMoE, self).__init__()
        self.num_experts = num_experts
        self.top_k = top_k
        self.reg_lambda = reg_lambda
        self.experts = nn.ModuleList([
            KANExpert(input_dim, hidden_dim, num_layers, num_classes, dropout, activation) 
            for _ in range(num_experts)
        ])
        self.gating_network = AttentionGatingNetwork(input_dim, num_experts, dropout)

    def forward(self, x):
        # Compute gating weights: shape [B, num_experts]
        gating_weights = self.gating_network(x)
        # Vectorize expert computation: Stack outputs from all experts.
        # Each expert returns logits of shape [B, num_classes].
        expert_outputs = torch.stack([expert(x) for expert in self.experts], dim=1)  # [B, num_experts, num_classes]
        
        # Optionally, select top_k experts for each sample.
        topk_vals, topk_idx = torch.topk(gating_weights, self.top_k, dim=1)  # both [B, top_k]
        # Create a mask for top_k experts.
        mask = torch.zeros_like(gating_weights)
        mask.scatter_(1, topk_idx, 1.0)
        # Zero out gating weights for experts not in top_k, then re-normalize.
        gating_weights = gating_weights * mask
        gating_weights = gating_weights / (gating_weights.sum(dim=1, keepdim=True) + 1e-8)
        
        # Multiply each expert's output by its gating weight and sum over experts.
        combined_logits = torch.sum(gating_weights.unsqueeze(2) * expert_outputs, dim=1)  # [B, num_classes]
        
        self.add_regularization_loss(gating_weights)
        return combined_logits

    def add_regularization_loss(self, gating_weights):
        mean_weights = gating_weights.mean(dim=0)
        load_balancing_loss = torch.std(mean_weights)
        entropy = -torch.sum(gating_weights * torch.log(gating_weights + 1e-8), dim=1).mean()
        self.regularization_loss = self.reg_lambda * (load_balancing_loss - entropy)



In [6]:
 #-----------------------------
# ViT_KANMoE Model integrating the ViT Backbone and KANMoE
# -----------------------------
class ViT_KANMoE(nn.Module):
    def __init__(self, num_classes, num_experts, hidden_dim, num_layers, top_k=2, reg_lambda=0.01, dropout=0.1, activation='GELU'):
        super(ViT_KANMoE, self).__init__()
        self.vit = vit_b_16(weights=ViT_B_16_Weights.IMAGENET1K_V1)
        self.vit.heads = nn.Identity()
        # Freeze all ViT layers except the last two for fine-tuning.
        for param in self.vit.parameters():
            param.requires_grad = False
        for param in self.vit.encoder.layers[-2:].parameters():
            param.requires_grad = True
        
        self.kan_moe = KANMoE(
            input_dim=768,
            hidden_dim=hidden_dim,
            num_layers=num_layers,
            num_experts=num_experts,
            num_classes=num_classes,
            top_k=top_k,
            reg_lambda=reg_lambda,
            dropout=dropout,
            activation=activation
        )

    # def forward(self, x):
    #     # Extract features via ViT. x shape: [B, C, H, W]
    #     features = self.vit(x)  # Output shape: [B, num_patches+1, 768]
    #     cls_features = features[:, 0, :]  # Use the CLS token: [B, 768]
    #     logits = self.kan_moe(cls_features)
    #     return logits
    

    def forward(self, x):
        features = self.vit(x)  # Check the actual shape
        # print("Feature shape:", features.shape)  # Debugging

        if len(features.shape) == 2:  # Shape: [batch_size, 768]
         cls_features = features 
          # Directly use it
        else:
            cls_features = features[:, 0, :]  # Extract CLS token
    
        logits = self.kan_moe(cls_features)
        return logits




In [7]:
# -----------------------------
# Example instantiation
# -----------------------------
num_classes = 7       # Classification for 7 classes
num_experts = 8       # Number of experts in MoE
hidden_dim = 256      # Hidden dimension for KAN layers
num_layers = 4        # Number of KAN layers per expert
top_k = 3             # Use top 3 experts per input
reg_lambda = 0.001    # Regularization strength
dropout = 0.2         # Dropout rate
activation = 'GELU'   # Choose 'GELU' or 'Mish'

model = ViT_KANMoE(num_classes, num_experts, hidden_dim, num_layers, top_k, reg_lambda, dropout, activation)
batch_size = 32

# Sample input for testing:
dummy_input = torch.randn(batch_size, 3, 224, 224)
output_logits = model(dummy_input)
print(output_logits.shape)  # Expected: [32, 7]


torch.Size([32, 7])


In [8]:
# Check which layers are trainable
for name, param in model.named_parameters():
    print(f"Layer: {name} | Trainable: {param.requires_grad}")

Layer: vit.class_token | Trainable: False
Layer: vit.conv_proj.weight | Trainable: False
Layer: vit.conv_proj.bias | Trainable: False
Layer: vit.encoder.pos_embedding | Trainable: False
Layer: vit.encoder.layers.encoder_layer_0.ln_1.weight | Trainable: False
Layer: vit.encoder.layers.encoder_layer_0.ln_1.bias | Trainable: False
Layer: vit.encoder.layers.encoder_layer_0.self_attention.in_proj_weight | Trainable: False
Layer: vit.encoder.layers.encoder_layer_0.self_attention.in_proj_bias | Trainable: False
Layer: vit.encoder.layers.encoder_layer_0.self_attention.out_proj.weight | Trainable: False
Layer: vit.encoder.layers.encoder_layer_0.self_attention.out_proj.bias | Trainable: False
Layer: vit.encoder.layers.encoder_layer_0.ln_2.weight | Trainable: False
Layer: vit.encoder.layers.encoder_layer_0.ln_2.bias | Trainable: False
Layer: vit.encoder.layers.encoder_layer_0.mlp.0.weight | Trainable: False
Layer: vit.encoder.layers.encoder_layer_0.mlp.0.bias | Trainable: False
Layer: vit.encoder

In [9]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(device)

cuda


In [10]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

# --- TensorBoard Setup ---
log_dir = "/teamspace/studios/this_studio/code2/ex1_5"
os.makedirs(log_dir, exist_ok=True)
writer = SummaryWriter(log_dir)

# --- Training Parameters ---
num_epochs = 200         # Total epochs
patience = 10            # Early stopping patience
best_val_loss = float("inf")
early_stopping_counter = 0
model_save_path = "exp1_5.pth"

# --- Optimizer Setup ---
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

# --- Learning Rate Scheduler ---
scheduler = CosineAnnealingLR(optimizer, T_max=10)

# --- Loss Function ---
criterion = nn.CrossEntropyLoss()

# --- Training Loop ---
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    epoch_loss = 0.0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

    for images, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)  # Forward pass: outputs shape [batch_size, num_classes]
        loss = criterion(outputs, labels)
        
        # Incorporate regularization loss from the MoE module
        total_loss = loss + model.kan_moe.regularization_loss
        
        total_loss.backward()  # Backward pass
        
        # Apply gradient clipping to stabilize training
        clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()  # Parameter update
        
        epoch_loss += total_loss.item()
        progress_bar.set_postfix(loss=total_loss.item())

    # Update the learning rate scheduler at the end of each epoch
    scheduler.step()

    avg_train_loss = epoch_loss / len(train_loader)
    writer.add_scalar("Loss/Train", avg_train_loss, epoch)

    # --- Validation Loop ---
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    writer.add_scalar("Loss/Validation", avg_val_loss, epoch)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")

    # --- Early Stopping & Model Saving ---
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stopping_counter = 0
        torch.save(model.state_dict(), model_save_path)  # Save best model
        print("✅ Best model saved!")
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= patience:
            print("⏹️ Early stopping triggered!")
            break  # Stop training

writer.close()
print("Training complete! 🎉")


2025-04-06 03:49:52.881020: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-06 03:49:52.943275: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-06 03:49:52.957019: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-06 03:49:53.066436: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-06 03:49:54.176463: W tensorflow/compiler/tf2

Epoch [1/200] - Train Loss: 1.6510, Val Loss: 0.5516
✅ Best model saved!


Epoch [2/200] - Train Loss: 0.8066, Val Loss: 0.5778


Epoch [3/200] - Train Loss: 0.6125, Val Loss: 0.1948
✅ Best model saved!


Epoch [4/200] - Train Loss: 0.5154, Val Loss: 0.1934
✅ Best model saved!


...

Epoch [22/200] - Train Loss: 0.3327, Val Loss: 0.4103


Epoch [23/200] - Train Loss: 0.2639, Val Loss: 0.2567


Epoch [24/200] - Train Loss: 0.3018, Val Loss: 0.2143


Epoch [25/200] - Train Loss: 0.2211, Val Loss: 0.2697


Epoch [26/200] - Train Loss: 0.1336, Val Loss: 0.1554
⏹️ Early stopping triggered!
Training complete! 🎉


In [11]:
#testing

In [12]:
# Evaluation
model.eval()
all_preds, all_labels, all_probs = [], [], [] 

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)

       
        # Forward pass
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)  # Convert to probabilities
        _, predicted = torch.max(outputs, 1)  # Get class predictions

        all_preds.extend(predicted.cpu().numpy())  # Store predictions
        all_labels.extend(labels.cpu().numpy())  # Store true labels
        all_probs.extend(probs.cpu().numpy())  # Store class probabilities







from sklearn.metrics import classification_report

# Convert lists to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_probs = np.array(all_probs)

# Compute Metrics
accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="weighted")
conf_matrix = confusion_matrix(all_labels, all_preds)
report=classification_report(all_labels,all_preds)


# Compute AUC (One-vs-Rest for multi-class)
# auc = roc_auc_score(all_labels, all_probs, multi_class="ovr")

print(f"Test Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
# print(f"AUC-ROC Score: {auc:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", report)




Test Accuracy: 97.04%
Precision: 0.9735
Recall: 0.9704
F1 Score: 0.9695
Confusion Matrix:
 [[21  0  0  0  0  0  0]
 [ 0 20  0  0  0  0  0]
 [ 0  0 23  0  0  0  0]
 [ 0  0  0 15  4  0  0]
 [ 0  0  0  0 34  0  0]
 [ 0  0  0  0  0  9  0]
 [ 0  0  0  0  0  0  9]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        21
           1       1.00      1.00      1.00        20
           2       1.00      1.00      1.00        23
           3       1.00      0.79      0.88        19
           4       0.89      1.00      0.94        34
           5       1.00      1.00      1.00         9
           6       1.00      1.00      1.00         9

    accuracy                           0.97       135
   macro avg       0.98      0.97      0.98       135
weighted avg       0.97      0.97      0.97       135



In [13]:
#training

In [14]:
# Evaluation
model.eval()
all_preds, all_labels, all_probs = [], [], [] 

with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

       
        # Forward pass
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)  # Convert to probabilities
        _, predicted = torch.max(outputs, 1)  # Get class predictions

        all_preds.extend(predicted.cpu().numpy())  # Store predictions
        all_labels.extend(labels.cpu().numpy())  # Store true labels
        all_probs.extend(probs.cpu().numpy())  # Store class probabilities







from sklearn.metrics import classification_report

# Convert lists to numpy arrays
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_probs = np.array(all_probs)

# Compute Metrics
accuracy = accuracy_score(all_labels, all_preds)
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="weighted")
conf_matrix = confusion_matrix(all_labels, all_preds)
report=classification_report(all_labels,all_preds)


# Compute AUC (One-vs-Rest for multi-class)
# auc = roc_auc_score(all_labels, all_probs, multi_class="ovr")

print(f"Training Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
# print(f"AUC-ROC Score: {auc:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", report)




Training Accuracy: 96.04%
Precision: 0.9612
Recall: 0.9604
F1 Score: 0.9604
Confusion Matrix:
 [[ 82   0   1   0   1   0   0]
 [  0  78   0   0   0   0   0]
 [  0   0  89   0   1   0   0]
 [  2   0   0  68   4   0   0]
 [  2   0   0   4 128   0   0]
 [  2   0   0   0   0  34   0]
 [  0   0   0   0   4   0  30]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.98      0.95        84
           1       1.00      1.00      1.00        78
           2       0.99      0.99      0.99        90
           3       0.94      0.92      0.93        74
           4       0.93      0.96      0.94       134
           5       1.00      0.94      0.97        36
           6       1.00      0.88      0.94        34

    accuracy                           0.96       530
   macro avg       0.97      0.95      0.96       530
weighted avg       0.96      0.96      0.96       530



In [16]:
print(model)


ViT_KANMoE(
  (vit): VisionTransformer(
    (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (encoder): Encoder(
      (dropout): Dropout(p=0.0, inplace=False)
      (layers): Sequential(
        (encoder_layer_0): EncoderBlock(
          (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (self_attention): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (dropout): Dropout(p=0.0, inplace=False)
          (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): MLPBlock(
            (0): Linear(in_features=768, out_features=3072, bias=True)
            (1): GELU(approximate='none')
            (2): Dropout(p=0.0, inplace=False)
            (3): Linear(in_features=3072, out_features=768, bias=True)
            (4): Dropout(p=0.0, inplace=False)
          )
        )
        (encoder_layer_1): EncoderBlock(
          (ln_1): LayerN